In [19]:
%matplotlib inline

In [20]:
# Import Dependencies
import json
import requests
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import scipy.stats as st


In [28]:
# Import csv file and make DataFrame
uber_data = Path("../Resources/Uber_Project_CSV.csv")
uber_df = pd.read_csv(uber_data)
uber_df=uber_df.astype({"pickup_dt":"datetime64[D]"})
uber_df.head()

,pickup_dt,borough,pickups,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,2015-01-01,Bronx,152,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
1,2015-01-01,Brooklyn,1519,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
2,2015-01-01,EWR,0,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
3,2015-01-01,Manhattan,5258,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
4,2015-01-01,Queens,405,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y


In [ ]:
#Import Pivot Table CSV file and make DataFrame
wind_uber= Path("../Resources/Pivot_Uber_Wind.csv")
wind_uber_df= pd.read_csv(wind_uber, encoding="UTF-8")
wind_uber_df


In [ ]:
#Import URL and API CSV file
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/new%2520york%2520city/2015-01-01/2015-06-30?elements=datetime%2CdatetimeEpoch%2Ctemp%2Ctempmax%2Ctempmin%2Cprecip%2Cwindspeed%2Cwindgust%2Cfeelslike%2Cfeelslikemax%2Cfeelslikemin%2Cpressure%2Cstations%2Cdegreedays%2Caccdegreedays&include=fcst%2Cobs%2Chistfcst%2Cstats%2Chours&key=9R8U63UTPUJVYD92WRCVMZZXB&options=preview&contentType=json"
weather_api = Path("../Resources/NYC_Weather_API_data.csv")
weather_df= pd.read_csv(weather_api)
weather_df.head()


In [29]:
# Does Wind Speed affect the number of pickups? How does wind with or without precipitation affect it? - Ryan
wind_df= uber_df[["pickup_dt", "borough", "pickups", "spd", "pcp01"]]
wind_df= wind_df.rename(columns={"pickup_dt": "Time of Observation"}|{"borough":"Borough"}|{"pickups":"Number of Pickups"}|{"spd":"Wind Speed"}|{"pcp01":"Precipitaion Amount"})

wind_df.head()


,Time of Observation,Borough,Number of Pickups,Wind Speed,Precipitaion Amount
0,2015-01-01,Bronx,152,5.0,0.0
1,2015-01-01,Brooklyn,1519,5.0,0.0
2,2015-01-01,EWR,0,5.0,0.0
3,2015-01-01,Manhattan,5258,5.0,0.0
4,2015-01-01,Queens,405,5.0,0.0


In [30]:
converted_wind_df= wind_df[["Borough", "Number of Pickups", "Wind Speed", "Precipitaion Amount", "Time of Observation"]]
converted_wind_df= converted_wind_df.groupby(["Borough"])
converted_wind_df.head()

,Borough,Number of Pickups,Wind Speed,Precipitaion Amount,Time of Observation
0,Bronx,152,5.0,0.0,2015-01-01
1,Brooklyn,1519,5.0,0.0,2015-01-01
2,EWR,0,5.0,0.0,2015-01-01
3,Manhattan,5258,5.0,0.0,2015-01-01
4,Queens,405,5.0,0.0,2015-01-01
5,Staten Island,6,5.0,0.0,2015-01-01
7,Bronx,120,3.0,0.0,2015-01-01
8,Brooklyn,1229,3.0,0.0,2015-01-01
9,EWR,0,3.0,0.0,2015-01-01
10,Manhattan,4345,3.0,0.0,2015-01-01


In [33]:
bronx_df= wind_df.loc[(wind_df["Borough"]=="Bronx"),:]
bronx_df= bronx_df.groupby("Time of Observation").agg({"Number of Pickups": "sum", "Wind Speed": "mean"}).reindex()
bronx_df.head()

,Number of Pickups,Wind Speed
Time of Observation,,
2015-01-01,1075,6.086957
2015-01-02,545,7.541667
2015-01-03,719,6.202083
2015-01-04,590,5.638194
2015-01-05,487,10.868056


In [ ]:
#Create dataframe for when precipitation occurs and for when there is no precipitation
wind_precip_df= wind_df.loc[(wind_df["Precipitaion Amount"]>0),:]
wind_precip_df.head()

In [ ]:
wind_dry_df= wind_df.loc[(wind_df["Precipitaion Amount"]==0),:]
wind_dry_df.head()


In [ ]:
#Does different boroughs? Ryan
borough_df= wind_df.groupby(["Borough"])

borough_df.head()
borough_df["Number of Pickups"].value_counts()

In [ ]:
# Correlation and Regression

## Analysis